In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from dask.distributed import Client
import dask
import xarray as xr
import xgcm
from dask.diagnostics import ProgressBar
import sys
import matplotlib.patches as patches
sys.path.append('/homes/metofac/wenegrat/xroms/')
sys.path.append('/homes/metofac/wenegrat/pyspec/')

#from xroms import open_roms_netcdf_dataset
import cmocean.cm as cmo
import xroms
from pyspec import spectrum as spec
from scipy import integrate as integrate
from timeit import default_timer as timer

#%%
plt.rcParams['text.usetex'] = True
plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['font.family'] = 'STIXGeneral'
plt.rcParams['font.size'] = 14
plt.rcParams['contour.negative_linestyle'] = 'solid'

import sys
sys.path.append("../src/")
import load_model_runs as lruns

from importlib import reload  
import time
from glob import glob


In [2]:
client = Client()
client

Client Scheduler: tcp://127.0.0.1:40956 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 64 Memory: 270.46 GB


In [3]:
reload(lruns)
dsOLD, gridOLD = lruns.loadRun('../data/model/jet_his_20_NC.nc', chunks={'s_rho':10, 's_w':10}, old=True)
dsOLD

chunks = {'time':1, 's_rho':5}
#dsWC, gridWC = lruns.loadZarrRun('../data/interim/dst.zarr')
dsWC, gridWC = lruns.loadRun('../data/model/JET_NML_CFB/', chunks=chunks, old=False)
#dsWC['f'] = dsOLD.f
dsWC



<xarray.Dataset>
Dimensions:     (auxil: 4, eta_rho: 4002, eta_v: 4001, ocean_time: 42, s_rho: 100, s_w: 101, xi_rho: 1002, xi_u: 1001)
Coordinates:
  * eta_rho     (eta_rho) int64 0 1 2 3 4 5 6 ... 3996 3997 3998 3999 4000 4001
  * eta_v       (eta_v) int64 0 1 2 3 4 5 6 ... 3995 3996 3997 3998 3999 4000
  * s_rho       (s_rho) float32 -0.995 -0.985 -0.975 ... -0.025 -0.015 -0.005
  * xi_rho      (xi_rho) int64 0 1 2 3 4 5 6 7 ... 995 996 997 998 999 1000 1001
  * xi_u        (xi_u) int64 0 1 2 3 4 5 6 7 ... 994 995 996 997 998 999 1000
  * s_w         (s_w) float32 -1.0 -0.99 -0.98 -0.97 ... -0.03 -0.02 -0.01 0.0
    x_rho       (eta_rho, xi_rho) float32 dask.array<chunksize=(4002, 1002), meta=np.ndarray>
    y_rho       (eta_rho, xi_rho) float32 dask.array<chunksize=(4002, 1002), meta=np.ndarray>
  * ocean_time  (ocean_time) float64 3.11e+07 3.119e+07 ... 3.456e+07 3.465e+07
    time        (ocean_time) float64 3.11e+07 3.119e+07 ... 3.456e+07 3.465e+07
    z_w         (ocean_time, s_w, eta_rho, xi_rho) float32 dask.array<chunksize=(1, 101, 4002, 1002), meta=np.ndarray>
    z_w_u       (ocean_time, s_w, eta_rho, xi_u) float32 dask.array<chunksize=(1, 101, 4002, 1001), meta=np.ndarray>
    z_w_v       (ocean_time, s_w, eta_v, xi_rho) float32 dask.array<chunksize=(1, 101, 4001, 1002), meta=np.ndarray>
    z_w_psi     (ocean_time, s_w, eta_v, xi_u) float32 dask.array<chunksize=(1, 101, 4001, 1001), meta=np.ndarray>
    z_rho       (ocean_time, s_rho, eta_rho, xi_rho) float32 dask.array<chunksize=(1, 100, 4002, 1002), meta=np.ndarray>
    z_rho_u     (ocean_time, s_rho, eta_rho, xi_u) float32 dask.array<chunksize=(1, 100, 4002, 1001), meta=np.ndarray>
    z_rho_v     (ocean_time, s_rho, eta_v, xi_rho) float32 dask.array<chunksize=(1, 100, 4001, 1002), meta=np.ndarray>
    z_rho_psi   (ocean_time, s_rho, eta_v, xi_u) float32 dask.array<chunksize=(1, 100, 4001, 1001), meta=np.ndarray>
    z_rho0      (s_rho, eta_rho, xi_rho) float32 -3.903e+03 ... -1.814
    z_rho_u0    (s_rho, eta_rho, xi_u) float32 -3.903e+03 -3.903e+03 ... -1.814
    z_rho_v0    (s_rho, eta_v, xi_rho) float32 -3.903e+03 -3.903e+03 ... -1.814
    z_rho_psi0  (s_rho, eta_v, xi_u) float32 -3.903e+03 -3.903e+03 ... -1.814
    z_w0        (s_w, eta_rho, xi_rho) float32 -4e+03 -4e+03 -4e+03 ... 0.0 0.0
    z_w_u0      (s_w, eta_rho, xi_u) float32 -4e+03 -4e+03 -4e+03 ... 0.0 0.0
    z_w_v0      (s_w, eta_v, xi_rho) float32 -4e+03 -4e+03 -4e+03 ... 0.0 0.0
    z_w_psi0    (s_w, eta_v, xi_u) float32 -4e+03 -4e+03 -4e+03 ... 0.0 0.0 0.0
Dimensions without coordinates: auxil
Data variables:
    spherical   |S1 ...
    xl          float32 ...
    el          float32 ...
    Vtransform  float32 ...
    sc_r        (s_rho) float32 dask.array<chunksize=(5,), meta=np.ndarray>
    sc_w        (s_w) float32 dask.array<chunksize=(101,), meta=np.ndarray>
    Cs_r        (s_rho) float32 ...
    Cs_w        (s_w) float32 ...
    hc          float32 ...
    h           (eta_rho, xi_rho) float32 4e+03 4e+03 4e+03 ... 4e+03 4e+03
    f           (eta_rho, xi_rho) float32 dask.array<chunksize=(4002, 1002), meta=np.ndarray>
    pm          (eta_rho, xi_rho) float32 0.002 0.002 0.002 ... 0.002 0.002
    pn          (eta_rho, xi_rho) float32 0.002 0.002 0.002 ... 0.002 0.002
    time_step   (ocean_time, auxil) int32 dask.array<chunksize=(1, 4), meta=np.ndarray>
    zeta        (ocean_time, eta_rho, xi_rho) float32 dask.array<chunksize=(1, 4002, 1002), meta=np.ndarray>
    u           (ocean_time, s_rho, eta_rho, xi_u) float32 dask.array<chunksize=(1, 5, 4002, 1001), meta=np.ndarray>
    v           (ocean_time, s_rho, eta_v, xi_rho) float32 dask.array<chunksize=(1, 5, 4001, 1002), meta=np.ndarray>
    temp        (ocean_time, s_rho, eta_rho, xi_rho) float32 dask.array<chunksize=(1, 5, 4002, 1002), meta=np.ndarray>
    w           (ocean_time, s_rho, eta_rho, xi_rho) float32 dask.array<chunksize=(1, 5, 4002, 1002), meta=np.ndarray>
    sustr       (ocean_time, eta_rho, xi_u) float32 dask.

In [4]:
#dsPV, gridPV = lruns.loadRun('../data/interim/pvfiles', chunks = chunks, old=False)

files = sorted(glob('../data/interim/pv_20_NC_extend/pv*.nc'))
dsPV = xr.open_mfdataset(files[1:], combine='nested',concat_dim='time', chunks={'s_rho':5})
dsPV

<xarray.Dataset>
Dimensions:  (eta_rho: 4002, s_rho: 100, time: 8, xi_rho: 1002)
Coordinates:
  * time     (time) float64 3.154e+07 3.197e+07 3.24e+07 ... 3.413e+07 3.456e+07
  * s_rho    (s_rho) float32 -0.995 -0.985 -0.975 ... -0.025 -0.015 -0.005
  * eta_rho  (eta_rho) int64 0 1 2 3 4 5 6 ... 3996 3997 3998 3999 4000 4001
  * xi_rho   (xi_rho) int64 0 1 2 3 4 5 6 7 ... 995 996 997 998 999 1000 1001
    x_rho    (eta_rho, xi_rho) float32 dask.array<chunksize=(4002, 1002), meta=np.ndarray>
    y_rho    (eta_rho, xi_rho) float32 dask.array<chunksize=(4002, 1002), meta=np.ndarray>
Data variables:
    ertel    (time, s_rho, eta_rho, xi_rho) float64 dask.array<chunksize=(1, 5, 4002, 1002), meta=np.ndarray>

In [5]:
# Volume integrate the PV
volPV = dsPV.ertel*dsWC.dx*dsWC.dy*dsWC.dz0

In [6]:
ny = np.array(dsWC.eta_rho.size)
nyc = int(ny/2)
nysp = int(ny/16)
yl = slice(nysp, ny-nysp)

zl = slice(0, 100)

yl

slice(250, 3752, None)

In [7]:
PV_SERIES = volPV.isel(eta_rho=yl, s_rho=zl).sum(('s_rho', 'eta_rho', 'xi_rho')).compute()

In [ ]:
plt.plot(PV_SERIES-PV_SERIES[0])

In [8]:
def calcIntPV(ds, grid, el=[0,-1], sl = [0,-1]):
    rhotob = -9.8/1020
    
    #vx = xroms.to_rho(grid.derivative(ds.v, 'X'),grid)
    vx = xroms.hgrad(
        ds.v,
        grid,
        which ='xi',
        hboundary='extend',
        hfill_value=None,
        sboundary = 'extend',
        sfill_value=None,
        hcoord='rho',
        scoord='s_rho')
    #uy = xroms.to_rho(grid.derivative(ds.u, 'Y'),grid)
    uy = xroms.hgrad(
        ds.u,
        grid,
        which ='eta',
        hboundary='extend',
        hfill_value=None,
        sboundary = 'extend',
        sfill_value=None, 
        hcoord='rho',
        scoord='s_rho')
    #vz = xroms.to_rho(grid.interp(grid.derivative(ds.v, 'Z', boundary='extend'),'Z',to='center', boundary='extend'),grid)
    #uz = xroms.to_rho(grid.interp(grid.derivative(ds.u, 'Z', boundary='extend'),'Z',to='center', boundary='extend'),grid)
    uz = xroms.ddz(
            ds.u,
            grid,
            hcoord='rho',
            scoord='s_rho',
            hboundary = 'extend',
            sboundary = 'extend',
            hfill_value= None,
            sfill_value= None)
    #termy = grid.integrate(((rhotob*ds.rho*uz).isel(eta_rho=-1) - (rhotob*ds.rho*uz).isel(eta_rho=0)), ('xi_rho','Z'))
    #termz = grid.integrate(((rhotob*ds.rho*(ds.f + vx - uy)).isel(s_rho=-1) 
    #                        - (rhotob*ds.rho*(ds.f + vx - uy)).isel(s_rho=0)), ('X', 'Y'))
    ei = el[-1]
    face1 = (rhotob*ds.rho.isel(eta_rho=ei)*uz.isel(eta_rho=ei)*ds.dz.isel(eta_rho=ei)*ds.dx.isel(eta_rho=ei)).sum(('xi_rho', 's_rho'))
    ei = el[0]
    face2 = (rhotob*ds.rho.isel(eta_rho=ei)*uz.isel(eta_rho=ei)*ds.dz.isel(eta_rho=ei)*ds.dx.isel(eta_rho=ei)).sum(('xi_rho', 's_rho'))
    termy = face1 - face2
    
    si = sl[-1]
    face1 = (rhotob*ds.rho.isel(s_rho=si)*(ds.f+vx.isel(s_rho=si)-uy.isel(s_rho=si))*ds.dx*ds.dy).isel(eta_rho=slice(el[0], el[-1])).sum(('xi_rho','eta_rho'))
    si = sl[0]
    face2 = (rhotob*ds.rho.isel(s_rho=si)*(ds.f+vx.isel(s_rho=si)-uy.isel(s_rho=si))*ds.dx*ds.dy).isel(eta_rho=slice(el[0], el[-1])).sum(('xi_rho','eta_rho'))

    termz = face1-face2
    return termy + termz

el = [250, 3750]
sl = [0, -1]
dst = dsWC
dst = dst.swap_dims({'ocean_time':'time'})
PVWC = calcIntPV(dst, gridWC, el=el, sl=sl)
PVWC

<xarray.DataArray (time: 42)>
dask.array<add, shape=(42,), dtype=float32, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
    ocean_time  (time) float64 3.11e+07 3.119e+07 ... 3.456e+07 3.465e+07
  * time        (time) float64 3.11e+07 3.119e+07 ... 3.456e+07 3.465e+07
Attributes:
    long_name:      potential temperature
    units:          Celsius
    field:          temperature, scalar, series
    standard_name:  sea_water_potential_temperature
    grid:           <xgcm.Grid>\nX Axis (not periodic, boundary=None):\n  * c...

In [ ]:
PV_SERIES_DIR = PVWC.isel(time = range(0, 40, 5)).compute()

distributed.nanny - WARNING - Worker process still alive after 3 seconds, killing
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker


In [ ]:
plt.plot(dst.time.isel(time=range(0, 40, 5)), PV_SERIES_DIR - PV_SERIES_DIR[1], marker='.')
plt.plot(dsPV.time, PV_SERIES-PV_SERIES[0], marker='.')

In [ ]:
buoyancy = xroms.buoyancy(dsWC.rho, rho0=dsWC.rho0)
plt.plot(buoyancy.isel(ocean_time=-1, eta_rho=1000, xi_rho=0), dsWC.z_rho0.isel(eta_rho=1000, xi_rho=0))

In [ ]:
plt.rcParams['text.usetex'] = False
dsPV.ertel.isel(time=0, xi_rho=0, s_rho=slice(50,100)).plot()

In [ ]:
yl


In [ ]:
plt.plot(buoyancy.isel(ocean_time=-1, s_rho=-1, xi_rho=0))
